## Long Range Transportation Plan

In [1]:
import _categorizing_utils as categorizing_utils
import _harmonization_utils as harmonization_utils
import _lrtp_utils as lrtp_utils
import _specific_list_utils as list_utils
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

In [2]:
import fsspec

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Ambag

In [5]:
ambag = lrtp_utils.ambag_lrtp()

In [6]:
lrtp_utils.give_info(ambag, "project", "project_description")

Citywide Bike Lanes                         2
The Alameda - Salinas Road Bike Route       2
Citywide Sidewalk Program                   2
Carmel to Pebble Beach Bike/Ped Facility    1
Second Street Bike Lane                     1
Name: project, dtype: int64
# of unique project titles: 277
After dropping duplicates using project and project_description: 280
Df shape: (280, 6)
Index(['ambag_id', 'project', 'project_description', 'total_cost_\n_$_000s_',
       'county', 'total_cost_millions'],
      dtype='object', name=0)


In [7]:
ambag.loc[ambag.project == "Citywide Bike Lanes"]

,ambag_id,project,project_description,total_cost_\n_$_000s_,county,total_cost_millions
133,MON-SOL075-SO,Citywide Bike Lanes,"Bike Lanes (2007 TIF M2, 2013 TIF M2); construct bike lanes citywide",1440,Santa Cruz,1440000
52,SC-SV-P41-SCV,Citywide Bike Lanes,Construction of additional bike lanes and paths citywide (including Green Hills).,2060,San Benito,2060000


### BCAG 

In [8]:
bcag = lrtp_utils.bcag_lrtp()

In [9]:
bcag[['fund_estimate','total_project_cost','rtp_id']].sample(3)

,fund_estimate,total_project_cost,rtp_id
35,$ 0.237 million,237000.00,36
69,$.75 million,750000.00,68
218,$0.005 million,4500.00,219


In [10]:
lrtp_utils.give_info(bcag, "project_descr", "total_project_cost")

New Traffic Signal                                                                                                                                   6
Turn lane capacity expansion, storage length expansion, channelization improvements, pedestrian safety due to increased traffic volumes.             4
East Gridley Rd. At Feather River, 1.0 mile east of Larkin Rd. Scope is to address cracks with a Methacrylate Deck treatment. Bridge No. 12C0022.    2
Roundabout (within existing ROW)                                                                                                                     2
operational flow improvments (traffic signals or roundabouts)                                                                                        2
Name: project_descr, dtype: int64
# of unique project titles: 239
After dropping duplicates using project_descr and total_project_cost: 247
Df shape: (250, 23)
Index(['rtp_id', 'agency', 'ftip', 'rtp', 'project_type', 'title',
       'project_de

In [11]:
# bcag.loc[bcag.project_descr == "Turn lane capacity expansion, storage length expansion, channelization improvements, pedestrian safety due to increased traffic volumes."]

### Fresno COG

In [12]:
fresno_cog = lrtp_utils.fresnocog_lrtp()

In [13]:
lrtp_utils.give_info(fresno_cog, "project_title", "estimated\ntotal_cost\n_$1,000_")

None                                               3
SR 41 at Elkhorn Ave Roundabout                    2
SR 41 at Mount Whitney Ave Roundabout              2
Ventura, SR 41 to SR 99  Widen from 4LU to 4 LD    2
Herndon-Polk to Weber : 4Ld to 6LD                 2
Name: project_title, dtype: int64
# of unique project titles: 3129
After dropping duplicates using project_title and estimated
total_cost
_$1,000_: 3143
Df shape: (3147, 10)
Index(['agency', 'project_id', 'project_title', 'project_description',
       'project_type', 'estimated_open_to_traffic',
       'estimated\ntotal_cost\n_$1,000_', 'est__o2t_year',
       'est_total_project_cost', 'financial_constraint'],
      dtype='object')


In [14]:
# fresno_cog.loc[fresno_cog.project_type.str.contains("Ac")]

In [15]:
lrtp_utils.group_duplicates(
    fresno_cog,
    "project_title",
    "estimated\ntotal_cost\n_$1,000_",
    "project_description",
)

,,project_description
project_title,"estimated total_cost _$1,000_",
"Ventura, SR 41 to SR 99 Widen from 4LU to 4 LD",3427.00,2
Herndon-Polk to Weber : 4Ld to 6LD,2931.00,2
10th Ave / Kamm Ave Traffic Signal,400.00,1
Millerton Road From Auberry Rd to SR 168 Shoulder Widening,5910.00,1
Midtown Trail: Millbrook - Michigan to McKinley,680.00,1
Milburn & Spruce: New Traffic Signal,620.00,1
Mill and Orange Park & Ride Lot,505.00,1
Millbrook & Nees: Left Turn Phasing,380.00,1
Millbrook & Olympic: New Traffic Signal,500.00,1


In [16]:
fresno_cog[fresno_cog.project_title.str.contains("Ventura, SR 41")]

,agency,project_id,project_title,project_description,project_type,estimated_open_to_traffic,"estimated\ntotal_cost\n_$1,000_",est__o2t_year,est_total_project_cost,financial_constraint
989,Fresno,FRE111312,"Ventura, SR 41 to SR 99 Widen from 4LU to 4 LD",Widen to 4 LN Divided Arterial,Streets & Roads - Capacity Increasing,2027-2031,3427.00,2028-01-01,3427000.00,constrained
2071,Fresno,FRE111312,"Ventura, SR 41 to SR 99 Widen from 4LU to 4 LD",Widen to 4 LN Divided Arterial(Measure C Project F in the Urban Regional Program),Streets & Roads-Capacity Increasing,2028,3427.00,NaT,NaN,constrained


In [17]:
3427*1_000

3427000

### KCAG

In [18]:
kcag = lrtp_utils.kcag_lrtp()

In [19]:
kcag.sample()

,category,jurisdiction,state_route,post_mile,location,project_limits,description,title,total_cost
24,Proposed Improvements,Lemoore,NaN,NaN,Redwood Drive,Kenwood Dr. to Lum Dr.,Overlay,None,0


In [20]:
# kcag.title.nunique()

In [21]:
lrtp_utils.give_info(kcag, "description", "category")

Overlay                                                                                        33
Widen from 2 to 4 lanes with left turns                                                         4
Reconstruct and improve curb/ramps                                                              3
Intersection Improvements                                                                       2
Overlay of various thicknesses.  Some shoulder work and dig-out and patch prior to overlay.     2
Name: description, dtype: int64
# of unique project titles: 18
After dropping duplicates using description and category: 20
Df shape: (84, 9)
Index(['category', 'jurisdiction', 'state_route', 'post_mile', 'location',
       'project_limits', 'description', 'title', 'total_cost'],
      dtype='object')


In [22]:
lrtp_utils.group_duplicates(kcag, "location", "description", "category")

,,category
location,description,
10th Avenue,Widen from 2 to 4 lanes with left turns,1
Purchase CNG Refuse Vehicle,Vehicle Purchase,1
Lemoore Avenue,"Widen to 5 lanes, add 2 lanes with continuous left turn lane",1
Linda Lee Circle,Overlay,1
Madera Street,Reconstruct and improve curb/ramps,1
Magnolia Street,Overlay,1
Major Roads,Overlay of various thicknesses. Some shoulder work and dig-out and patch prior to overlay.,1
Merced Street,Reconstruct and improve curb/ramps,1
Minor Roads,Overlay of various thicknesses. Some shoulder work and dig-out and patch prior to overlay.,1


In [23]:
kcag.total_cost.describe()

count   84.00
mean     0.00
std      0.00
min      0.00
25%      0.00
50%      0.00
75%      0.00
max      0.00
Name: total_cost, dtype: float64

In [24]:
kcag.sample()

,category,jurisdiction,state_route,post_mile,location,project_limits,description,title,total_cost
41,Proposed Improvements,Lemoore,NaN,NaN,Washington Drive,Lemoore Ave. to B St.,Overlay,None,0


### KernCog

In [25]:
kern = lrtp_utils.kern_lrtp()

In [26]:
kern.sample(2)

,project_title,scope,yoe_w__new_revenue,yoe_w_o_new_reven,maint__inflation_savings,cost
661,Bakersfield ‐ Class III Bike Route,Mezzadro/Alderbrk/Lavina from Allen Road to Allen Road ‐ 3.63 miles,NaN,NaN,NaN,NaN
311,West Corridor‐North,17th Standard Rd to Rt 99 ‐extend freeway,75583000,100000000,24417000,75583000


In [27]:
lrtp_utils.give_info(kern, "project_title", "cost")

Kern County ‐ Class II Bike Ln        85
Bakersfield ‐ Class III Bike Blvd     85
Bakersfield ‐ Class II Bike Ln        78
Kern County ‐ Sidewalk Improvement    71
Kern County ‐ Class III Bike Blvd     35
Name: project_title, dtype: int64
# of unique project titles: 371
After dropping duplicates using project_title and cost: 1180
Df shape: (1411, 6)
Index(['project_title', 'scope', 'yoe_w__new_revenue', 'yoe_w_o_new_reven',
       'maint__inflation_savings', 'cost'],
      dtype='object')


In [28]:
# kern.loc[kern.project_title == "M St / Belshaw St‐Park St"]

In [29]:
lrtp_utils.group_duplicates(kern, "project_title", "scope", "cost")

,,cost
project_title,scope,
County Areas ‐ transit,"Senior/disabled & advanced technology transit, vanpools, shared ride, aviation",9
County Areas ‐ active transportation,"Safe complete streets, pedestrian enhancements",5
County Areas ‐ bike facilities,"Construct Class I (trails), II (lanes) or Class III (routes) Bike Paths; striping; signage",3
Kern County ‐ Complete Streets/ITS I,Other Future developments funded by a transportation impact fee and mitigation,3
Kern County ‐ Corridor Improvement,Hall Rd (San Emidio St to Habecker Rd) .5 mi.,2
Route 58,2Future Rt 58 from I‐5 to Heath Rd at Stockdale Hwy ‐ construct new freeway,2
Kern County ‐ Corridor Improvement,Myrtle Av (Panama Ln to Wharton Av) .9 mi.,2
Caltrans ‐ Class II Bike Ln,E Bear Mtn Blvd SR 223 (Comanche Dr to Union St) 12.6 mi.,2
Santa Fe Way,1Hageman to Los Angeles Ave ‐ widen to four lanes,2


In [30]:
kern[kern.project_title == "US 395"].sort_values(["cost"])

,project_title,scope,yoe_w__new_revenue,yoe_w_o_new_reven,maint__inflation_savings,cost
155,US 395,Between Rt 178 and China Lake Blvd ‐ construct passing lanes,15117000,191000000,4883000,15117000
161,US 395,San Bdo County Line to Rt 14 ‐ widen to four lanes,184422000,451035000,59578000,184422000


In [31]:
kern[kern.project_title == "Teh. Willow Springs Rd"].sort_values(["cost"])

,project_title,scope,yoe_w__new_revenue,yoe_w_o_new_reven,maint__inflation_savings,cost
1115,Teh. Willow Springs Rd,Rt 58 to Rosamond Blvd ‐ widen to four lanes,114101000,150961000,36861000,114101000
1322,Teh. Willow Springs Rd,Rt 58 to Rosamond Blvd ‐ widen to four lanes,377914000,377914000,122086000,377914000


### Madera CTC

In [32]:
madera = lrtp_utils.madera_lrtp()

In [33]:
lrtp_utils.group_duplicates(madera, "project_name", "total_cost", "description")

,,description
project_name,total_cost,
Purchase Transit Vehicle,300000,3
Road 36,563000,3
Avenue 12,10000000,3
Various,1000000,3
Sunset Avenue & N Westberry Blvd,$,2
Pecan Avenue,665000,2
Avenue 9,567000,2
James Madison Elementary School SRTS Pedestrian Improvements,$,2
Northwest Downtown Pedestrian Access,$,2


In [34]:
lrtp_utils.give_info(madera, "project_name", "total_cost")

Various                            20
Avenue 12                          10
Avenue 9                            8
State Route 41                      6
Purchase Transit Van (Electric)     5
Name: project_name, dtype: int64
# of unique project titles: 515
After dropping duplicates using project_name and total_cost: 735
Df shape: (765, 8)
Index(['category', 'agency', 'project_name', 'location', 'project_id',
       'description', 'opening_year', 'total_cost'],
      dtype='object')


In [35]:
madera[madera.project_name == "Purchase Transit Vehicle"]

,category,agency,project_name,location,project_id,description,opening_year,total_cost
2,Public Transit,City of Madera,Purchase Transit Vehicle,NaN,MAD213201,Purchase new transit vehicle,NaN,300000
3,Public Transit,City of Madera,Purchase Transit Vehicle,NaN,MAD213202,Purchase new transit vehicle,NaN,300000
4,Public Transit,City of Madera,Purchase Transit Vehicle,NaN,MAD213203,Purchase new transit vehicle,NaN,300000
23,Public Transit,Madera County,Purchase Transit Vehicle,NaN,MAD115006,Purchase new transit vehicle,NaN,825000
24,Public Transit,Madera County,Purchase Transit Vehicle,NaN,MAD115010,Purchase new transit vehicle,NaN,57000


In [36]:
madera[madera.project_name == "Avenue 12"].head()

,category,agency,project_name,location,project_id,description,opening_year,total_cost
46,Streets and Roads,Madera County,Avenue 12,Road 38 to Road 40,NaN,2 Lanes to 4 Lanes,2027.00,10000000
47,Streets and Roads,Madera County,Avenue 12,Road 40 to Riverwalk\nBlvd,NaN,2 Lanes to 6 Lanes,2027.00,10000000
48,Streets and Roads,Madera County,Avenue 12,Riverwalk Blvd to SR\n41,NaN,4 Lanes to 8 Lanes,2027.00,10000000
49,Streets and Roads,Madera County,Avenue 12,SR 41 To Flagbarn Rd,NaN,2 Lanes to 4 Lanes,2030.00,4250000
51,Streets and Roads,Madera County,Avenue 12,Road 30 1/2 to Road 36,NaN,2 Lanes to 4 Lanes,2030.00,21000000


### MCAGOV

In [37]:
mcagov = lrtp_utils.mcagov_lrtp()

In [38]:
lrtp_utils.give_info(mcagov, "title", "total_cost\n_$1,000s_")

Bellevue Rd Realignment/Reconstruction    1
Minturn Road Rehabilitation               1
MCAG Vanpool Program (dibs)               1
Measure V Transit Free Ride Program       1
Planada Route Improvements                1
Name: title, dtype: int64
# of unique project titles: 108
After dropping duplicates using title and total_cost
_$1,000s_: 108
Df shape: (108, 8)
Index(['agency', 'title', 'limits_description', 'type', 'completion\nyear',
       'total_cost\n_$1,000s_', 'funding_sources', 'total_cost_millions'],
      dtype='object')


In [39]:
mcagov.sample()

,agency,title,limits_description,type,completion\nyear,"total_cost\n_$1,000s_",funding_sources,total_cost_millions
15,Gustine,Borelli Ranch Park Multi-use Path,Construct a Multi-use Path from Fentem Rd to the end of Via Palermo,Active (Bike/Ped),2030,450,"CMAQ, Local, Measure V",450000


### MTC
* 133 projects but point and line map returns 240 ish rows.
* The geospatial projects are split into various phases while the regular pandas df is just the project in its entirety.
* https://www.planbayarea.org/2050-plan/final-plan-bay-area-2050/final-supplemental-reports/interactive-transportation-project-list
* https://opendata.mtc.ca.gov/datasets/MTC::2023-transportation-improvement-program-projects-point/about
* https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2050-transportation-projects-line/about
* https://data.bayareametro.gov/Projects/Plan-Bay-Area-2050-Transportation-Project-List/y7ka-jksz

In [40]:
mtc_og = lrtp_utils.mtc_lrtp()

In [41]:
# mtc_og.drop(columns = ['geometry','scope']).sample()

In [42]:
#layer_list = ["planbayarea2050_rtp_line", "planbayarea2050_rtp_point"]
#url_pt_1 = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/"
#url_pt_2 = "/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
#gdf = lrtp_utils.open_rest_server(url_pt_1, url_pt_2, layer_list)

In [43]:
#gdf.columns

In [44]:
#gdf.loc[gdf.rtp_id == "21-T10-088"]

In [45]:
#all_projects_url = "https://data.bayareametro.gov/resource/y7ka-jksz.json?$query=SELECT%0A%20%20%60plan_strategy%60%2C%0A%20%20%60rtpid%60%2C%0A%20%20%60title%60%2C%0A%20%20%60scope%60%2C%0A%20%20%60open_period%60%2C%0A%20%20%60funding_millions_yoe%60%2C%0A%20%20%60county%60"
#all_projects = pd.read_json(all_projects_url)

In [46]:
#all_projects.loc[all_projects.rtpid == "21-T10-088"]

In [47]:
#mtc_og.loc[mtc_og.rtpid == "21-T10-088"].drop(columns = ['scope','geometry'])

In [48]:
#mtc_og.loc[mtc_og.proj_title == "Light Rail | Service Expansion | VTA | Stevens Creek Blvd"].drop(columns = ['scope','geometry'])

### Sacramento Area Council of Governments (SACOG)
* https://www.sacog.org/sites/main/files/file-attachments/2020_mtp-scs.pdf?1580330993

In [49]:
sacog = lrtp_utils.sacog_lrtp()

96 rows are headers


In [50]:
lrtp_utils.give_info(sacog, "title", "description")

Safety Improvements                            2
Connect Card Implementation                    2
Aux Lane Project: EB Latrobe Road              1
Bus Maintenance Facility #1 Rehabilitation     1
Enhanced Employee Protection Warning System    1
Name: title, dtype: int64
# of unique project titles: 1599
After dropping duplicates using title and description: 1601
Df shape: (1601, 10)
Index(['id', 'status__planned,_programmed_or_project_development_only_',
       'county', 'lead_agency', 'budget_category', 'title', 'description',
       'total_project_cost__2018_dollars_',
       'year_of_expenditure_cost_for_planned_projects', 'completion_timing'],
      dtype='object')


In [51]:
sacog.sample()

,id,"status__planned,_programmed_or_project_development_only_",county,lead_agency,budget_category,title,description,total_project_cost__2018_dollars_,year_of_expenditure_cost_for_planned_projects,completion_timing
59,ELD19224,Planned,ELD,El Dorado County,B- Road & Highway Capacity,Bass Lake Road Widening,"Widen and reconstruct Bass Lake Road from US 50 to Serrano Parkway to 4- lane divided road. Includes a median, sidewalk and bike lanes. (CIP66109)",14257000,17370770,2026-2030


### SANDAG
* https://www.arcgis.com/home/item.html?id=ba8b2a6247ef463cb7411f039959f3ee
* https://www.arcgis.com/apps/mapviewer/index.html?webmap=ba8b2a6247ef463cb7411f039959f3ee

In [52]:
sandag = lrtp_utils.sandag_lrtp()

In [53]:
lrtp_utils.group_duplicates(sandag, "project_name", "description", "total_project_cost")

,,total_project_cost
project_name,description,
LRT 530,Green Line (Santee to Downtown; Double/Third-tracking and Grade Separations),2
Rapid 630,Iris Trolley/Palomar to Kearny Mesa via I-5/163 and City College,1
I-5 (Cassidy St to Harbor Dr.),6 Freeway Lanes and 4 Managed Lanes,1
Rapid 471,Downtown Escondido to East Escondido,1
Rapid 450 Phase 2,Oceanside to Escondido via Palomar Airport Rd and SR 78 (Full version of Rapid),1
Rapid 450 Phase 1,Oceanside to Escondido via Palomar Airport Rd and SR 78 (Light version of Rapid),1
Rapid 440,Carlsbad to Escondido Transit Center via Palomar Airport Rd,1
Rapid 41,Fashion Valley to UTC/UC San Diego via Linda Vista and Clairemont,1
Rapid 30,"Balboa Station to Sorrento Mesa via Pacific Beach, La Jolla, UTC",1


In [54]:
sandag.loc[sandag.project_name == "LRT 530"][
    ["project_name", "total_project_cost", "description"]
]

,project_name,total_project_cost,description
159,LRT 530,192000000.00,Green Line (Santee to Downtown; Double/Third-tracking and Grade Separations)
160,LRT 530,192000000.00,Green Line (Santee to Downtown; Double/Third-tracking and Grade Separations)


### SBCAG

In [55]:
sbcag = lrtp_utils.sbcag_lrtp()

65 rows are headers


In [56]:
lrtp_utils.give_info(sbcag, "project_title", "total_cost__$000s_")

SB-PL-15: Upper De la Vina St Gap Closure and Safe Crossings              2
CT-1: SR 246 Passing Lanes – Planting Mitigation (FTIP CT93)(EA 0C641)    1
SM-PL-10: A Street Widening                                               1
Sol-PL-6: Fredensborg Canyon Rd/Adobe Creek Culvert Replacement           1
Sol-PL-5: Solvang School Sidewalk Project                                 1
Name: project_title, dtype: int64
# of unique project titles: 418
After dropping duplicates using project_title and total_cost__$000s_: 419
Df shape: (419, 8)
Index(['type', 'project_title', 'phase', 'description',
       'primary_funding_source_s_', 'year', 'total_cost__$000s_',
       'total_cost_millions'],
      dtype='object')


In [57]:
sbcag.sample()

,type,project_title,phase,description,primary_funding_source_s_,year,total_cost__$000s_,total_cost_millions
169,Programmed,MTD-12: Terminal 2 - Interim\nReactivation,Capital,Reactivate Terminal 2 in Goleta as a\nsecondary bus yard,NaN,2021,1498,1498000


### Shasta County Regional Transportation Planning Agency (SCRTPA)

In [58]:
scrtpa = lrtp_utils.scrtpa_lrtp()

In [59]:
lrtp_utils.give_info(scrtpa, "regional_transportation_projects", "cost")

E Cypress Ave-Alfreda Way-Victor Ave                      2
GROVE ST-B ST-WALNUT ST                                   2
Churn Creek Rd-Rancho Rd-Churn Creek Rd/S Bonnyview Rd    2
Hartnell Ave-Victor Ave-Shasta View Dr                    2
Hawley Rd-Hawley Rd (e/o Norwich Ct)-Collyer Dr           2
Name: regional_transportation_projects, dtype: int64
# of unique project titles: 1051
After dropping duplicates using regional_transportation_projects and cost: 1066
Df shape: (1066, 7)
Index(['regional_transportation_projects',
       'short_term_total_est_cost_of_project',
       'long_term_total_est_cost_of_project', 'project_band',
       'project_type\n_project_intent_', 'expected_funding_sources', 'cost'],
      dtype='object')


In [60]:
scrtpa[
    scrtpa.regional_transportation_projects == "E Cypress Ave-Alfreda Way-Victor Ave"
]

,regional_transportation_projects,short_term_total_est_cost_of_project,long_term_total_est_cost_of_project,project_band,project_type\n_project_intent_,expected_funding_sources,cost
831,E Cypress Ave-Alfreda Way-Victor Ave,NaN,29913,(2026-2040),Buffered Bike Lane,unknown,29913
833,E Cypress Ave-Alfreda Way-Victor Ave,NaN,529884,(2026-2040),Sidewalk,unknown,529884


### San Luis Obispo Council of Governments (SLOCOG)

In [61]:
slocog = lrtp_utils.slocog_lrtp()

In [62]:
lrtp_utils.give_info(slocog, "project_name", "total_cost")

Hazel Ln. SRTS Improvements                          2
Main St. Sidewalks                                   2
Los Berros Rd. Widening                              2
Octagon Barn Park-and-Ride Lot and shared-use lot    1
Augusta Neighborhood SRTS Improvements               1
Name: project_name, dtype: int64
# of unique project titles: 416
After dropping duplicates using project_name and total_cost: 420
Df shape: (420, 12)
Index(['index', 'project_name', 'project_description', 'sponsor', 'community',
       'time_period', 'project_type', '_2023_rtp_project_id', 'time_horizon',
       'total_cost', 'project_id', 'geometry'],
      dtype='object')


In [63]:
# slocog[slocog.project_name == "Hazel Ln. SRTS Improvements"].sample()

### Southern California Association of Governments (SCAG) 

In [64]:
scag = lrtp_utils.scag_lrtp()

360 rows are headers


In [65]:
lrtp_utils.give_info(scag, "description", "route_name")

WIDEN FROM 2 TO 4 LANES      170
WIDEN FROM 4 TO 6 LANES       64
WIDEN FROM 2 TO 6 LANES       43
CONSTRUCT 4 LANE ARTERIAL     21
WIDEN FROM 3 TO 4 LANES       10
Name: description, dtype: int64
# of unique project titles: 2607
After dropping duplicates using description and route_name: 2824
Df shape: (2952, 13)
Index(['system', 'lead_agency', 'rtp_id', 'route_#', 'route_name', 'from',
       'to', 'description', 'completion_year', 'project_cost__$1,000s_',
       'county', 'project_title', 'project_cost_millions'],
      dtype='object')


In [66]:
lrtp_utils.group_duplicates(
    scag, "description", "project_cost__$1,000s_", "project_title"
)

project_title
description             project_cost__$1,000s_               
WIDEN FROM 2 TO 4 LANES 3000                                5
                        6000                                3
                        3210                                2
                        1878                                2
WIDEN FROM 4 TO 6 LANES 10000                               2
WIDEN FROM 2 TO 4 LANES 6901                                2
                        8000                                2
WIDEN FROM 4 TO 6 LANES 5000                                2
                        20000                               2
WIDEN FROM 2 TO 4 LANES 9571                                2

In [67]:
3000*1_000

3000000

In [68]:
scag.loc[
    (scag.description == "WIDEN FROM 4 TO 6 LANES")
    & (scag["project_cost__$1,000s_"] == 20000)
]

,system,lead_agency,rtp_id,route_#,route_name,from,to,description,completion_year,"project_cost__$1,000s_",county,project_title,project_cost_millions
1441,LOCAL HIGHWAY,PERRIS,3A01WT105,0,PERRIS BLVD,RAMONA EXPWY,NUEVO RD,WIDEN FROM 4 TO 6 LANES,2025,20000,NaN,No Title,20000000
1595,LOCAL HIGHWAY,"RIVERSIDE, CITY OF",3A07315,0,VAN BUREN BLVD,AUDREY AVE,GARFIELD,WIDEN FROM 4 TO 6 LANES,2026,20000,NaN,No Title,20000000


In [69]:
scag.loc[scag.description.str.contains("Rail Capital Projects", case=False)]


,system,lead_agency,rtp_id,route_#,route_name,from,to,description,completion_year,"project_cost__$1,000s_",county,project_title,project_cost_millions
812,TRANSIT,LOS ANGELES COUNTY MTA (METRO),1TL0703,0,METRO RAIL TRANSIT CAPITAL,COUNTYWIDE,NaN,RAIL CAPITAL PROJECTS,2040,19151000,NaN,No Title,19151000000


In [70]:
38960000*1_000

38960000000

###  SJCOG

In [71]:
sjcog = lrtp_utils.sjcog_lrtp()

In [72]:
lrtp_utils.give_info(sjcog, "project_description", "total")

Widen from 2 to 6 lanes                        9
Install ramp meters and ITS elements           7
Rehabilitation of various streets and roads    7
Widen from 2 to 4 lanes                        6
Reconstruct interchange                        5
Name: project_description, dtype: int64
# of unique project titles: 208
After dropping duplicates using project_description and total: 252
Df shape: (262, 16)
Index(['_2018_rtp_mpo_id', 'ctips_id_#', 'jurisdiction', 'facility_name_route',
       'project_description', 'project_limits', 'total', 'ftip_programming',
       'nepa_approval', 'open_to_traffic', 'completion', 'completion_date',
       'facility_name_route_1', 'project_description_1', 'milestone_years_1',
       'project_name'],
      dtype='object')


In [73]:
lrtp_utils.group_duplicates(sjcog, "project_description", "total", "project_name")

,,project_name
project_description,total,
Widen from 6 to 8 lanes (inside/outside),90000000.00,3
Modification of existing interchange - ENVIRONMENTAL ONLY,2500000.00,2
Reconstruct interchange,30000000.00,2
Install ramp meters and ITS elements,23400000.00,2
Widen from 2 to 6 lanes,9000000.00,2
Widen 6 to 8 lanes (inside),90000000.00,2
Rehabilitation of various streets and roads,197236934.00,1
"Reconstruct interchange to provide 6 through lanes on SR 99, 4 lanes on Harney between Reynolds Ranch Pkwy and SR 99 and modify on-ramps and off-ramps",35000000.00,1
Reconstruct interchange to provide operational and safety improvements on SR 99 at Turner Road (PM 31.3/31.6),6331338.00,1


In [74]:
sjcog.loc[
    sjcog.project_description == "Widen from 6 to 8 lanes (inside/outside)"
].head()

,_2018_rtp_mpo_id,ctips_id_#,jurisdiction,facility_name_route,project_description,project_limits,total,ftip_programming,nepa_approval,open_to_traffic,completion,completion_date,facility_name_route_1,project_description_1,milestone_years_1,project_name
1,SJ07-1003,NaN,Caltrans,I-205 Managed Lanes,Widen from 6 to 8 lanes (inside/outside),Alameda County Line to Eleventh Street,90000000.00,NaN,2022.00,2028.00,NaN,NaN,NaN,NaN,NaN,None
2,SJ14-1001,NaN,Caltrans,I-205 Managed Lanes,Widen from 6 to 8 lanes (inside/outside),Eleventh Street to MacArthur Drive,90000000.00,NaN,2022.00,2028.00,NaN,NaN,NaN,NaN,NaN,None
3,SJ14-1002,NaN,Caltrans,I-205 Managed Lanes,Widen from 6 to 8 lanes (inside/outside),MacArthur Drive to I-5,90000000.00,NaN,2022.00,2028.00,NaN,NaN,NaN,NaN,NaN,None


In [75]:
sjcog[~sjcog.completion_date.isna()].sample()

,_2018_rtp_mpo_id,ctips_id_#,jurisdiction,facility_name_route,project_description,project_limits,total,ftip_programming,nepa_approval,open_to_traffic,completion,completion_date,facility_name_route_1,project_description_1,milestone_years_1,project_name
176,SJ14-6006,NaN,SJRRC,Robert J. Cabral Station Expansion,"Construct park and ride lot and related on-street parking, sidewalks, lighting, security, and other passenger amenity improvements","In Stockton, between the UPRR, Weber Avenue, Union Street, and Main Street",1263387.00,NaN,NaN,NaN,NaN,2020.00,NaN,NaN,NaN,None


### Stancog

In [76]:
stancog = lrtp_utils.stancog_lrtp()

68 rows are headers


In [77]:
stancog.columns

Index(['jurisdiction', 'location', 'project_limits', 'description',
       'total_cost', 'open_to_traffic', 'funding_source', 'system_preserv_',
       'capacity_enhance_', 'safety', 'oper_', 'complete_streets',
       'active\ntransporta_tion', 'transit', 'other', 'title'],
      dtype='object', name=0)

In [78]:
lrtp_utils.give_info(stancog, "description", "total_cost")

Intersection Improvements               19
Install Traffic Signal                  15
Widen from 2 to 4 lanes                  8
Shoulder Widening - Class 2 Bikepath     7
Widen to 3 lanes                         7
Name: description, dtype: int64
# of unique project titles: 417
After dropping duplicates using description and total_cost: 519
Df shape: (552, 16)
Index(['jurisdiction', 'location', 'project_limits', 'description',
       'total_cost', 'open_to_traffic', 'funding_source', 'system_preserv_',
       'capacity_enhance_', 'safety', 'oper_', 'complete_streets',
       'active\ntransporta_tion', 'transit', 'other', 'title'],
      dtype='object', name=0)


In [79]:
lrtp_utils.group_duplicates(stancog, "project_limits", "description", "total_cost")

,,total_cost
project_limits,description,
Intersection,Install Signal & Intersection Improvements,3
Main Street to T Street,"Pavement Rehabilitation, Maintenance",3
Various Locations,Pedestrian Improvements,2
Driskell to Sherman Parkway,"Pavement Rehabilitation, Maintenance, and complete streets improvements",2
Highway 33 to Canal School Rd,Install Collector Street improvements,2
Yolo St to Jensen Rd,Bicycle and Pedestrian Improvements,2
SR-108 at First Street,Install Congestion Management improvements,1
SR-108 at Claus,Install signal light at Claus & SR-108 and Install congestion Management Improvements at First Street & SR-108,1
SR-108 to Riverbank City Limits (Near Oakdale Rd),Multi-Use Path (Class 1) and pedestrian improvements. (Non-Motorized Transportation Plan Top 25: Route 22),1


In [80]:
stancog.loc[(stancog.description == "Install Traffic Signal")
]

,jurisdiction,location,project_limits,description,total_cost,open_to_traffic,funding_source,system_preserv_,capacity_enhance_,safety,oper_,complete_streets,active\ntransporta_tion,transit,other,title
0,Ceres,Morgan Rd,Service Rd & Morgan Rd,Install Traffic Signal,347800,2022,"PFF, CMAQ",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
4,Ceres,Crows Landing Rd,Crows Landing Rd & A Street,Install Traffic Signal,430500,2025,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
6,Ceres,Hatch Rd,Hatch Rd & Faith Home Rd,Install Traffic Signal,600000,2025,"CMAQ, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
9,Ceres,Crows Landing Rd,Crows Landing Rd & Grayson Rd,Install Traffic Signal,499100,2040,"CMAQ, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
10,Ceres,Service Road,Service Road & Ustick,Install Traffic Signal,499100,2035,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
11,Ceres,Whitmore Ave,Whitmore Ave. @ E Street,Install Traffic Signal,499100,2030,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
12,Ceres,Whitmore Ave,Whitmore Ave & Boothe Rd,Install Traffic Signal,514000,2025,"CMAQ, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
13,Ceres,Whitmore Ave,Whitmore Ave. @ Knox Rd,Install Traffic Signal,545300,2030,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
16,Ceres,Crows Landing Rd,Crows Landing Rd & B Street,Install Traffic Signal,578500,2030,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None
17,Ceres,Ustick Rd,Ustick Rd & F Street,Install Traffic Signal,578500,2030,"WLSP, PFF",NaN,NaN,X,X,NaN,NaN,NaN,NaN,None


### TMPO (Tahoe)
* Open data portal: https://www.tahoeopendata.org/search?tags=transportation
* https://www.tahoeopendata.org/datasets/TRPA::2020-regional-transportation-plan-projects/about

In [81]:
tahoe = lrtp_utils.tmpo_lrtp()

### TCAG Tulare County Association of Governments (TCAG)

In [82]:
tcag = lrtp_utils.tcag_lrtp()

In [83]:
tcag.groupby(["project_title", "cost_x_1,000"]).agg(
    {"project_category": "count"}
).sort_values("project_category", ascending=False).head()

,,project_category
project_title,"cost_x_1,000",
Bike/Ped Improvements next 9 years,0.00,6
Bike/Ped Improvements next 9 yrs second half of measure R,500.00,3
Avenue 316 Construct new roadway,6500.00,3
Transit Center Minor,300.00,2
Bike/Ped Improvements next 9 years,390.00,2


In [84]:
lrtp_utils.group_duplicates(
    tcag, "project_title", "project_description", "cost_x_1,000"
)

cost_x_1,000
project_title                                             project_description                                                                           
Bike/Ped                                                  Various                                                                                      2
Santa Fe Trail Regional                                   Santa Fe Trail Regional                                                                      2
Purchase Buses                                            Fleet replacement, expansion and electrification, 2036-2046                                  2
                                                          Fleet replacement, expansion and electrification, 2022-2035                                  2
Bike/Ped Improvements next 9 yrs second half of measure R used $49,000 for alta heritage; $50K used for El Monte Way,                                  2
Bike/Ped Improvements next 9 years                        Used $100,000 and $200,000 to Alta Heritage and Roosevelt Paseo respectively                 2
SR 198 Construct new I/C                                  SR-198 at Road 148 (Tower) Construct new interchange                                         1
SR 198 Intersection Improvements                          SR-198 at Spruce Rd Roundabout and local street improvements                                 1
SR 198 Operational  I/C improve.                          SR-198 at Lovers Lane Intersection, rehab, operational improvements                          1
                                                          SR-198 downtown corridor interchanges Turn lane, intersection, ramp improvements             1

In [85]:
tcag.loc[
    (tcag.project_title == "Santa Fe Trail Regional")
    & (tcag.project_description == "Santa Fe Trail Regional")
]

,project_id__if_applicable_,jurisdiction,project_title,project_description,project_category,completed_by_2035_y_n,local,regional,state,federal,"cost_x_1,000",completion_year,cost_in_millions
49,NaN,Visalia (Regional),Santa Fe Trail Regional,Santa Fe Trail Regional,Active Transportation,Y,NaN,X,NaN,NaN,586.37,2035,586370.00
50,NaN,TCAG,Santa Fe Trail Regional,Santa Fe Trail Regional,Active Transportation,Y,NaN,X,NaN,NaN,2413.63,2035,2413630.00


## Stack

### Add Lost

In [86]:
all_mpo_df, all_mpo_gdf = lrtp_utils.all_mpo(True)

96 rows are headers
65 rows are headers
360 rows are headers
68 rows are headers


In [87]:
# los_df = harmonization_utils.load_lost()

In [88]:
# los_df.sample()

In [89]:
len(all_mpo_df)

16276

In [90]:
all_mpo_gdf.shape

(1355, 10)

In [91]:
all_mpo_gdf.data_source.unique()

array(['Bcag Rtp/Scs Published In 2020 Lrtp',
       'Mtc Plan Bay Area 2050 Lrtp',
       'Sandag Rtp/Scs Published 2021 Lrtp',
       'Slocog 2023-2045 Regional Transportation Plan Lrtp',
       'Tmpo Rtp/Scs Published In 2020 Lrtp'], dtype=object)

In [92]:
all_mpo_df.shape

(16276, 10)

In [93]:
all_mpo_df.data_source.value_counts()

Fresno Rtp/Scs Published In 2022 Lrtp                                                      3147
Scag Rtp/Scs Published In 2020 Lrtp                                                        2952
Lost                                                                                       1849
Sacog Rtp/Scs Published In 2019 Lrtp                                                       1601
Kern 2022 Rtp/Scs Lrtp                                                                     1411
Scrtpa Rtp/Scs Published In 2018 Lrtp                                                      1066
Madera 2046 Rtp/Scs Lrtp                                                                    765
Stancog Rtp/Scs Published In 2022 Lrtp                                                      552
Slocog 2023-2045 Regional Transportation Plan Lrtp                                          420
Sbcag Connected 2050 Lrtp                                                                   419
Sandag Rtp/Scs Published 2021 Lrtp      

In [94]:
# all_mpo_df.drop(columns = ['notes']).sample(3)

In [95]:
gcs_test = pd.read_excel("gs://calitp-analytics-data/data-analyses/project_list/LRTP/all_LRTP_LOST.xlsx")

In [96]:
gcs_test.sample()

,project_title,lead_agency,project_year,project_description,total_project_cost,city,county,data_source,notes
2419,Temperance - Belmont To Dakota: 2 Lu To 6 Ld,Fresno Rtp/Scs Published In 2022,2032-2036,"Widen From 2 Lu To 6 Ld With Bike Lanes, Trail, Sidewalks, Curb And Gutter",15340000.00,None,None,Fresno Rtp/Scs Published In 2022 Lrtp,"Project Type: Bike & Ped, Streets & Roads - Capacity Increasing, Financial Constraint: Constrained"


In [97]:
"""
all_mpo_gdf.explore(
    "project_title",
    cmap="tab20c",
    height=400,
    width=600,
    style_kwds={"weight": 4},
    legend=False,
)
"""

'\nall_mpo_gdf.explore(\n    "project_title",\n    cmap="tab20c",\n    height=400,\n    width=600,\n    style_kwds={"weight": 4},\n    legend=False,\n)\n'

### Cordon Pricing

In [98]:
congest_keywords = [
    "congestion pricing",
    "variable tolls",
    "express lane",
    "value pricing",
    "rush hour",
    "cordon",
    "dynamic pricing",
    "dynamically  priced",
    "high occupancy",
    "mobility pricing",
    "occupancy",
    "toll lane",
    "performance pricing",
    "peak travel",
    "managed lane",
    "tollway",
    "express toll",
    "fixed pricing",
    "hot lane",
    "hov lane",
    "expressed toll lane",
]

In [99]:
cordon_gdf, cordon_df = list_utils.filter_projects(
    all_mpo_df,
    [
        "project_title",
        "project_description",
    ],
    congest_keywords,
    "congestion_pricing_lrtp_lost",
    True,
)

In [100]:
len(cordon_df), len(cordon_gdf)

(61, 7)

In [101]:
len(cordon_df) / len(all_mpo_df)

0.003747849594494962

In [102]:
cordon_df.project_title.nunique(), cordon_gdf.project_title.nunique()

(33, 5)

In [103]:
cordon_df.groupby(["data_source"]).agg({"project_title": "nunique"})

,project_title
data_source,
Fresno Rtp/Scs Published In 2022 Lrtp,1
Lost,4
Mtc Plan Bay Area 2050 Lrtp,2
Sacog Rtp/Scs Published In 2019 Lrtp,22
Sandag Rtp/Scs Published 2021 Lrtp,1
Scag Rtp/Scs Published In 2020 Lrtp,1
Slocog 2023-2045 Regional Transportation Plan Lrtp,1
Tmpo Rtp/Scs Published In 2020 Lrtp,1


In [104]:
gdf_map_subset = [
    "project_title",
    "lead_agency",
    "project_year",
    "total_project_cost",
    "geometry",
]

In [105]:
"""
cordon_gdf[gdf_map_subset].explore(
    cmap="tab20c",
    height=400,
    width=600,
    style_kwds={"weight": 4},
    legend=False,
)
"""

'\ncordon_gdf[gdf_map_subset].explore(\n    cmap="tab20c",\n    height=400,\n    width=600,\n    style_kwds={"weight": 4},\n    legend=False,\n)\n'

### Categorizing

In [106]:
all_mpo_df = all_mpo_df.fillna("None")

In [107]:
categorized_df = categorizing_utils.add_categories(all_mpo_df)

In [108]:
drop_cols = [
    "lower_case_project_title",
    "lower_case_project_description",
    "lower_case_project_title_keyword_search",
    "lower_case_project_description_keyword_search",
    "geometry",
    "city",
    "county",
]

In [109]:
categorized_df = categorized_df.drop(columns=drop_cols)

In [110]:
categorized_df.n_categories.value_counts()

0.00    7351
1.00    5056
2.00    2605
3.00     877
4.00     297
5.00      71
6.00      19
Name: n_categories, dtype: int64

In [111]:
all_projects_categorized_by_mpo = categorizing_utils.find_categories_by_mpo(categorized_df)

In [112]:
all_projects_categorized_by_mpo.head()

,data_source,category,total_projects
0,Ambag 2050 Metropolitan Transportation Plan & The Sustainable Communities Strategy Lrtp,Active Transp,235.00
1,Ambag 2050 Metropolitan Transportation Plan & The Sustainable Communities Strategy Lrtp,Bridge,10.00
2,Ambag 2050 Metropolitan Transportation Plan & The Sustainable Communities Strategy Lrtp,Congestion Relief,0.00
3,Ambag 2050 Metropolitan Transportation Plan & The Sustainable Communities Strategy Lrtp,Freeway,9.00
4,Ambag 2050 Metropolitan Transportation Plan & The Sustainable Communities Strategy Lrtp,Infra Resiliency Er,39.00


In [113]:
categorizing_utils.count_all_categories(categorized_df)

,Category,Total Projects
0,Active Transp,3140.00
1,Transit,1612.00
2,Bridge,769.00
3,Street,4737.00
4,Freeway,733.00
5,Infra Resiliency Er,2820.00
6,Congestion Relief,108.00
7,Passenger Mode Shift,111.00
8,Safety,524.00
